In [10]:
import pandas as pd

In [11]:
df_full = pd.DataFrame()

for year in range(2011, 2024):
    header = 9

    df = pd.read_excel(f'../data/docentes/dsu_municipios_{year}.xlsx', header=[header])

    columns = []
    columns_rename = {}

    if year < 2015:
        columns = ['ano', 'NO_REGIAO', 'SIGLA', 'NO_MUNICIPIO', 'TIPOLOCA', 'Dependad', 'DSU_INF',
                   'DSU_PRE', 'DSU_CRE']
        columns_rename = {'ano': 'ANO', 'NO_REGIAO': 'REGIAO', 'SIGLA': 'UF', 'Dependad': 'TP_DEPENDENCIA',
                            'DSU_INF': 'PER_DOC_CUR_SUP_INF', 'DSU_CRE': 'PER_DOC_CUR_SUP_INF_CRE',
                            'DSU_PRE': 'PER_DOC_CUR_SUP_INF_PRE', 'TIPOLOCA': 'TP_CATEGORIA'}
    elif year == 2015:
        columns = ['NU_ANO_CENSO', 'NO_REGIAO', 'SG_UF', 'NO_MUNICIPIO', 'TIPOLOCA', 'Dependad', 'DSU_INF',
                    'DSU_PRE', 'DSU_CRE']
        columns_rename = {'NU_ANO_CENSO': 'ANO', 'NO_REGIAO': 'REGIAO', 'SG_UF': 'UF', 'Dependad': 'TP_DEPENDENCIA',
                            'DSU_INF': 'PER_DOC_CUR_SUP_INF', 'DSU_CRE': 'PER_DOC_CUR_SUP_INF_CRE',
                            'DSU_PRE': 'PER_DOC_CUR_SUP_INF_PRE', 'TIPOLOCA': 'TP_CATEGORIA'}
    elif year < 2018:
        columns = ['NU_ANO_CENSO', 'NO_REGIAO', 'SG_UF', 'CO_MUNICIPIO', 'TIPOLOCA', 'Dependad', 'DSU_INF',
                   'DSU_PRE', 'DSU_CRE']
        columns_rename = {'NU_ANO_CENSO': 'ANO', 'NO_REGIAO': 'REGIAO', 'SG_UF': 'UF', 'Dependad': 'TP_DEPENDENCIA',
                            'DSU_INF': 'PER_DOC_CUR_SUP_INF', 'DSU_CRE': 'PER_DOC_CUR_SUP_INF_CRE',
                            'DSU_PRE': 'PER_DOC_CUR_SUP_INF_PRE', 'TIPOLOCA': 'TP_CATEGORIA',
                            'CO_MUNICIPIO': 'NO_MUNICIPIO'}
    else:
        columns = ['NU_ANO_CENSO', 'NO_REGIAO', 'SG_UF', 'NO_MUNICIPIO', 'NO_CATEGORIA', 'NO_DEPENDENCIA',
                   'ED_INF_CAT_0', 'CRE_CAT_0', 'PRE_CAT_0']
        columns_rename = {'NU_ANO_CENSO': 'ANO', 'NO_REGIAO': 'REGIAO', 'SG_UF': 'UF',
                          'NO_DEPENDENCIA': 'TP_DEPENDENCIA','ED_INF_CAT_0': 'PER_DOC_CUR_SUP_INF',
                          'CRE_CAT_0': 'PER_DOC_CUR_SUP_INF_CRE', 'PRE_CAT_0': 'PER_DOC_CUR_SUP_INF_PRE',
                          'NO_CATEGORIA': 'TP_CATEGORIA'}

    df = df[columns]
    df.rename(columns=columns_rename, inplace=True)

    with open('../data/cities.txt', 'r') as fd:
        cities = fd.read().splitlines()

    if year > 2014:
        df = df[df['NO_MUNICIPIO'].isin(cities)]
    else:
        df = df[df['NO_MUNICIPIO'].isin([city.upper() for city in cities])]
        dict_cities = {city.upper(): city for city in cities}
        df['NO_MUNICIPIO'] = df['NO_MUNICIPIO'].map(dict_cities)
    df.reset_index(drop=True, inplace=True)

    df_full = pd.concat([df_full, df])

df_full = df_full[df_full['NO_MUNICIPIO'].isin(['Itabuna', 'Ilhéus'])].reset_index(drop=True)

df_full.to_csv(f'../output/dsu_municipios.csv', index=False)
df_full.head()

,ANO,REGIAO,UF,NO_MUNICIPIO,TP_CATEGORIA,TP_DEPENDENCIA,PER_DOC_CUR_SUP_INF,PER_DOC_CUR_SUP_INF_PRE,PER_DOC_CUR_SUP_INF_CRE
0,2011,Nordeste,BA,Itabuna,Total,Total,46.5,42.9,57.3
1,2011,Nordeste,BA,Itabuna,Total,Estadual,--,--,--
2,2011,Nordeste,BA,Itabuna,Total,Municipal,64,62.5,68
3,2011,Nordeste,BA,Itabuna,Total,Privada,21.1,19,36
4,2011,Nordeste,BA,Itabuna,Total,Publico,64,62.5,68


In [12]:
with pd.ExcelWriter("../output/full_data.xlsx", mode='a') as writer:
    df_full.to_excel(writer, sheet_name='dsu_municipios', index=False)